In [50]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

Load in the scouting df from the ``get_szn_data.ipynb`` output

In [56]:
szn = '2024-25'
df = pd.read_csv(f'data/scouting/scouting_{szn}.csv')
df.head()

,name,prev_position,team,xP,xA,xG,assists,goals_scored,bonus,bps,minutes,total_points,prev_price,GW,current_price,current_position
0,Jack Hinshelwood,MID,Brighton,1.5,0.00,0.00,0,0,0,0,0,0,45,1,50.0,MID
1,Jadon Sancho,MID,Man Utd,3.0,0.05,0.00,0,0,0,4,22,1,70,1,65.0,MID
2,Vitaly Janelt,MID,Brentford,2.1,0.01,0.02,0,0,0,6,90,2,55,1,50.0,MID
3,Ionuț Radu,GK,Bournemouth,2.4,0.00,0.00,0,0,0,0,0,0,45,1,NaN,DEF
4,Jack Grealish,MID,Man City,4.1,0.00,0.00,0,0,0,0,0,0,75,1,65.0,MID


## Select a gamweek window (optional)

Let's select only the last half of the season, as that may be more indicative for current player performance. 

This collapses the the data across all selected GWs!

In [62]:
def select_gw_subset(df, gw_start, gw_end):
    '''
    Function to select a subset of the FDR dataframe based on the gameweeks and teams you want to look at.
    
    Parameters:
    - df: merged GW DataFrame - each row is the stats for a player in a given GW.
    - gw_start: The starting gameweek (inclusive).
    - gw_end: The ending gameweek (inclusive).
    
    Returns:
    - A DataFrame subset based on the specified gameweeks.
    '''
    gws = np.arange(gw_start,gw_end+1)
    df = df[df['GW'].isin(gws)].reset_index(drop=True)
    df = df.drop(columns=['GW'])

    df = df.dropna()


    df = df.groupby(by=['name', 'current_position', 'team'], as_index=False).agg(xP={'xP':'sum'},
                                                            xA=('xA', 'sum'),
                                                            xG=('xG','sum'),
                                                            A=('assists', 'sum'),
                                                            G=('goals_scored', 'sum'),
                                                            mins=('minutes', 'sum'),
                                                            total_points=('total_points', 'sum'),
                                                            bonus=('bonus','sum'),
                                                            prev_price=('prev_price', 'mean'),
                                                            current_price=('current_price', 'mean'))


    # players['value'] = players['value'].round(0).astype(int)
    # players
    
    return df


In [63]:
df_subset = select_gw_subset(df, 19, 38)
df_subset.head()

TypeError: Must provide 'func' or tuples of '(column, aggfunc).

## Points per 90, per $

Let's add some informative columns

In [ ]:
# points per minute
df_subset['ppm'] = df_subset['points'] / df_subset['mins']

# number of 90s (full matches) they played
df_subset['90s'] = df_subset['mins'] / 90

#
df_subset['ppg'] = df_subset['points'] / df_subset['90s']

## Overperforming and underperforming players (Watkins)

# OLD

### Create a players df

In [37]:
players = df.groupby(by=['name', 'position', 'team'], as_index=False).agg(xa=('expected_assists', 'sum'),
                                                              A=('assists', 'sum'),
                                                              xg=('expected_goals','sum'),
                                                              G=('goals_scored', 'sum'),
                                                              mins=('minutes', 'sum'),
                                                              points=('total_points', 'sum'),
                                                              bonus=('bonus','sum'),
                                                              value=('value', 'mean'))
                                                             
players['value'] = players['value'].round(0).astype(int)
players

,name,position,team,xa,A,xg,G,mins,points,bonus,value
0,Aaron Cresswell,DEF,West Ham,1.09260,1,0.1433,0,2235,74,8,48
1,Aaron Hickey,DEF,Brentford,0.46000,3,0.3100,0,1918,63,2,49
2,Aaron Ramsdale,GK,Arsenal,0.01619,0,0.0000,0,3420,143,9,49
3,Aaron Wan-Bissaka,DEF,Man Utd,0.91981,0,1.1857,0,1432,67,3,43
4,Abdoulaye Doucouré,MID,Everton,1.46968,2,2.4918,5,1527,76,10,53
...,...,...,...,...,...,...,...,...,...,...,...
680,Yerson Mosquera Valdelamar,DEF,Wolves,0.00000,0,0.0000,0,0,0,0,39
681,Yoane Wissa,MID,Brentford,0.93299,4,3.0141,7,1587,111,6,53
682,Yves Bissouma,MID,Spurs,0.19400,0,0.0528,0,998,27,0,47
683,Zack Steffen,GK,Man City,0.00000,0,0.0000,0,0,0,0,40


In [38]:
#drop those who played less than 350 mins
# players = players[players['mins'] > 350]
# players

In [39]:
#sort values by total points 
players = players.sort_values(by=['points'], ascending=False)
players = players.reset_index()
players = players.drop(columns='index')
players

,name,position,team,xa,A,xg,G,mins,points,bonus,value
0,Erling Haaland,FWD,Man City,1.33591,9,17.4943,36,2767,272,40,121
1,Harry Kane,FWD,Spurs,2.72175,9,12.0402,30,3406,263,48,116
2,Mohamed Salah,MID,Liverpool,4.69210,13,14.7355,19,3290,239,23,129
3,Martin Ødegaard,MID,Arsenal,5.11981,8,6.7737,15,3132,212,30,67
4,Marcus Rashford,MID,Man Utd,1.59559,7,11.4763,17,2880,205,21,69
...,...,...,...,...,...,...,...,...,...,...,...
680,Nuno Varela Tavares,DEF,Arsenal,0.00000,0,0.0000,0,0,0,0,45
681,Ishé Samuels-Smith,DEF,Everton,0.00000,0,0.0000,0,0,0,0,40
682,Isaac Price,MID,Everton,0.01821,0,0.0000,0,32,0,0,45
683,Imari Samuels,DEF,Brighton,0.00000,0,0.0000,0,0,0,0,40


In [40]:
#Get names and prices of 2023 season
position_mapping = {
        1: "GK",
        2: "DEF",
        3: "MID",
        4: "FWD"
    }
# Send a GET request to the FPL website
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
response = requests.get(url)
data = response.json()

# Extract player information from the response
all_players = data['elements']

# Iterate over each player and print their name and price
names_2023 = []
positions_2023 = []
prices_2023 = []

for player in all_players:
    fname = player['first_name']
    lname = player['second_name']
    name = fname+' '+lname
    #get price at the start of the 2023 season
    price = (player['now_cost'] - player['cost_change_start']) / 10  # Divide by 10 to get the actual price
    #get 2023 position
    element_type = player['element_type']
    position = position_mapping[element_type]
    #append to lists
    positions_2023.append(position)
    names_2023.append(name)
    prices_2023.append(price)

In [41]:
#get names in both 2022 and 2023 szn
names_2022 = players['name'].unique()

ns = []
poss = []
ps = []

for i in range(len(names_2023)):
    if names_2023[i] in names_2022:
        ns.append(names_2023[i])
        poss.append(positions_2023[i])
        ps.append(prices_2023[i])
        
#name and price, and name and position mappings
name_price = dict(zip(ns, ps))
name_pos = dict(zip(ns, poss))

In [42]:
#get price from 2023 game
players['2023_price'] = [''] * players.shape[0]
players['position'] = [''] * players.shape[0]

for i in range(players.shape[0]):
    name = players['name'].iloc[i]
    try:
        price = name_price[name]
    except:
        price = ''
    try:
        position = name_pos[name]
    except:
        position = ''
    #players['2023_price'].iloc[i] = price
    players.loc[i, '2023_price'] = price
    players.loc[i, 'position'] = position
players = players.drop(columns='value')
players['ppm'] = players['points'] / players['mins']
players['90s'] = players['mins'] / 90
players['ppg'] = players['points'] / players['90s']
players.head(20)

,name,position,team,xa,A,xg,G,mins,points,bonus,2023_price,ppm,90s,ppg
0,Erling Haaland,FWD,Man City,1.33591,9,17.4943,36,2767,272,40,14.0,0.098301,30.744444,8.847127
1,Harry Kane,FWD,Spurs,2.72175,9,12.0402,30,3406,263,48,12.5,0.077217,37.844444,6.949501
2,Mohamed Salah,MID,Liverpool,4.69210,13,14.7355,19,3290,239,23,12.5,0.072644,36.555556,6.537994
3,Martin Ødegaard,MID,Arsenal,5.11981,8,6.7737,15,3132,212,30,8.5,0.067688,34.800000,6.091954
4,Marcus Rashford,MID,Man Utd,1.59559,7,11.4763,17,2880,205,21,9.0,0.071181,32.000000,6.406250
5,Bukayo Saka,MID,Arsenal,4.19346,12,6.8705,14,3183,202,19,8.5,0.063462,35.366667,5.711593
6,Gabriel Martinelli Silva,MID,Arsenal,4.22851,9,6.1959,15,2789,198,18,8.0,0.070993,30.988889,6.389387
7,Kieran Trippier,DEF,Newcastle,7.73913,9,0.4675,1,3342,198,39,6.5,0.059246,37.133333,5.332136
8,Kevin De Bruyne,MID,Man City,7.41061,18,2.9174,7,2413,183,26,10.5,0.075839,26.811111,6.825528
9,Ivan Toney,FWD,Brentford,2.60493,4,11.7113,20,2953,182,35,8.0,0.061632,32.811111,5.546901


## Top Goalies

In [43]:
goalies = players[players['position'] == 'GK']
goalies.head(30)

,name,position,team,xa,A,xg,G,mins,points,bonus,2023_price,ppm,90s,ppg
12,David Raya Martin,GK,Brentford,0.10840,0,0.11,0,3420,166,20,5.0,0.048538,38.000000,4.368421
13,Alisson Ramses Becker,GK,Liverpool,0.02541,1,0.00,0,3330,162,15,5.5,0.048649,37.000000,4.378378
18,Nick Pope,GK,Newcastle,0.00363,0,0.00,0,3261,157,14,5.5,0.048145,36.233333,4.333027
25,José Malheiro de Sá,GK,Wolves,0.00303,0,0.00,0,3240,148,18,5.0,0.045679,36.000000,4.111111
29,Aaron Ramsdale,GK,Arsenal,0.01619,0,0.00,0,3420,143,9,5.0,0.041813,38.000000,3.763158
32,Bernd Leno,GK,Fulham,0.00921,0,0.00,0,3240,142,17,4.5,0.043827,36.000000,3.944444
39,Emiliano Martínez Romero,GK,Aston Villa,0.21244,0,0.00,0,3139,135,15,5.0,0.043007,34.877778,3.870659
48,Lukasz Fabianski,GK,West Ham,0.00290,0,0.00,0,3111,127,10,4.5,0.040823,34.566667,3.674060
55,Jordan Pickford,GK,Everton,0.08292,1,0.00,0,3330,124,10,4.5,0.037237,37.000000,3.351351
60,Ederson Santana de Moraes,GK,Man City,0.00434,1,0.00,0,3150,121,8,5.5,0.038413,35.000000,3.457143


In [44]:
goalies = goalies[goalies['2023_price']<=4.5]
goalies

,name,position,team,xa,A,xg,G,mins,points,bonus,2023_price,ppm,90s,ppg
32,Bernd Leno,GK,Fulham,0.00921,0,0.0,0,3240,142,17,4.5,0.043827,36.000000,3.944444
48,Lukasz Fabianski,GK,West Ham,0.00290,0,0.0,0,3111,127,10,4.5,0.040823,34.566667,3.674060
55,Jordan Pickford,GK,Everton,0.08292,1,0.0,0,3330,124,10,4.5,0.037237,37.000000,3.351351
98,Vicente Guaita,GK,Crystal Palace,0.00900,0,0.0,0,2430,99,10,4.5,0.040741,27.000000,3.666667
112,Norberto Murara Neto,GK,Bournemouth,0.02269,0,0.0,0,2385,93,9,4.5,0.038994,26.500000,3.509434
137,Dean Henderson,GK,Nott'm Forest,0.00654,0,0.0,0,1620,82,12,4.5,0.050617,18.000000,4.555556
139,Robert Sánchez,GK,Brighton,0.00782,0,0.0,0,2070,82,10,4.5,0.039614,23.000000,3.565217
185,Jason Steele,GK,Brighton,0.15000,1,0.0,0,1350,65,8,4.5,0.048148,15.000000,4.333333
266,Fraser Forster,GK,Spurs,0.00007,0,0.0,0,1215,40,1,4.0,0.032922,13.500000,2.962963
281,Sam Johnstone,GK,Crystal Palace,0.00000,0,0.0,0,810,36,5,4.5,0.044444,9.000000,4.000000


## Top Defenders

In [45]:
defs = players[players['position'] == 'DEF']
defs.head(20)

,name,position,team,xa,A,xg,G,mins,points,bonus,2023_price,ppm,90s,ppg
7,Kieran Trippier,DEF,Newcastle,7.73913,9,0.4675,1,3342,198,39,6.5,0.059246,37.133333,5.332136
20,Trent Alexander-Arnold,DEF,Liverpool,7.02897,11,1.3384,2,2922,156,21,8.0,0.053388,32.466667,4.804928
21,Benjamin White,DEF,Arsenal,2.81544,5,0.9776,2,3054,156,12,5.5,0.051081,33.933333,4.597250
27,Gabriel dos Santos Magalhães,DEF,Arsenal,0.47026,0,2.8539,3,3409,146,15,5.0,0.042828,37.877778,3.854503
30,Ben Mee,DEF,Brentford,0.38631,2,1.9479,3,3269,143,11,5.0,0.043744,36.322222,3.936984
36,Fabian Schär,DEF,Newcastle,1.25849,3,3.7244,1,3207,139,6,5.0,0.043343,35.633333,3.900842
44,Tyrone Mings,DEF,Aston Villa,0.40507,3,0.9400,1,3150,130,17,4.5,0.041270,35.000000,3.714286
45,Sven Botman,DEF,Newcastle,0.18437,2,2.4064,0,3127,129,8,4.5,0.041254,34.744444,3.712824
46,Dan Burn,DEF,Newcastle,1.14651,0,1.7445,1,3109,129,6,4.5,0.041492,34.544444,3.734320
47,Pervis Estupiñán,DEF,Brighton,2.51978,7,1.6653,1,2674,128,15,5.0,0.047868,29.711111,4.308153


In [46]:
defs[defs['team'] == 'Man City']

,name,position,team,xa,A,xg,G,mins,points,bonus,2023_price,ppm,90s,ppg
111,John Stones,DEF,Man City,0.27697,2,0.7877,2,1844,93,13,5.5,0.050434,20.488889,4.539046
115,Nathan Aké,DEF,Man City,0.99391,0,1.4433,1,1871,92,9,5.0,0.049172,20.788889,4.425441
134,Manuel Akanji,DEF,Man City,0.86633,1,0.6545,0,2284,82,5,5.0,0.035902,25.377778,3.231173
142,Rúben Gato Alves Dias,DEF,Man City,0.22000,0,0.7500,0,1997,80,1,5.5,0.040060,22.188889,3.605408
162,João Cancelo,DEF,Man City,0.48951,2,0.3074,2,1273,73,10,6.0,0.057345,14.144444,5.161037
172,Kyle Walker,DEF,Man City,1.09699,0,0.0743,0,1953,69,0,5.0,0.035330,21.700000,3.179724
260,Aymeric Laporte,DEF,Man City,0.36248,0,0.3845,0,992,40,3,5.0,0.040323,11.022222,3.629032
304,Rico Lewis,DEF,Man City,1.61457,0,0.2094,0,900,29,0,4.5,0.032222,10.000000,2.900000
342,Sergio Gómez,DEF,Man City,0.86000,1,0.2300,0,340,21,1,4.0,0.061765,3.777778,5.558824


## Top Midfielders

In [47]:
mid = players[players['position'] == 'MID']
mid.head(20)

,name,position,team,xa,A,xg,G,mins,points,bonus,2023_price,ppm,90s,ppg
2,Mohamed Salah,MID,Liverpool,4.69210,13,14.7355,19,3290,239,23,12.5,0.072644,36.555556,6.537994
3,Martin Ødegaard,MID,Arsenal,5.11981,8,6.7737,15,3132,212,30,8.5,0.067688,34.800000,6.091954
4,Marcus Rashford,MID,Man Utd,1.59559,7,11.4763,17,2880,205,21,9.0,0.071181,32.000000,6.406250
5,Bukayo Saka,MID,Arsenal,4.19346,12,6.8705,14,3183,202,19,8.5,0.063462,35.366667,5.711593
6,Gabriel Martinelli Silva,MID,Arsenal,4.22851,9,6.1959,15,2789,198,18,8.0,0.070993,30.988889,6.389387
8,Kevin De Bruyne,MID,Man City,7.41061,18,2.9174,7,2413,183,26,10.5,0.075839,26.811111,6.825528
10,Bruno Borges Fernandes,MID,Man Utd,8.00037,9,7.3556,8,3317,176,23,8.5,0.053060,36.855556,4.775399
15,Pascal Groß,MID,Brighton,6.32062,8,2.4868,9,3240,159,14,6.5,0.049074,36.000000,4.416667
16,Eberechi Eze,MID,Crystal Palace,1.57310,5,5.2418,10,2631,159,21,6.5,0.060433,29.233333,5.438997
17,Miguel Almirón Rejala,MID,Newcastle,1.86317,4,3.4971,11,2487,158,15,6.5,0.063530,27.633333,5.717732


## Top Forwards

In [48]:
fwd = players[players['position'] == 'FWD']
fwd.head(50)

,name,position,team,xa,A,xg,G,mins,points,bonus,2023_price,ppm,90s,ppg
0,Erling Haaland,FWD,Man City,1.33591,9,17.4943,36,2767,272,40,14.0,0.098301,30.744444,8.847127
1,Harry Kane,FWD,Spurs,2.72175,9,12.0402,30,3406,263,48,12.5,0.077217,37.844444,6.949501
9,Ivan Toney,FWD,Brentford,2.60493,4,11.7113,20,2953,182,35,8.0,0.061632,32.811111,5.546901
11,Ollie Watkins,FWD,Aston Villa,1.06980,8,11.8067,15,3129,175,25,8.0,0.055928,34.766667,5.033557
19,Callum Wilson,FWD,Newcastle,1.54261,6,10.2789,18,1870,157,20,8.0,0.083957,20.777778,7.556150
43,Dominic Solanke,FWD,Bournemouth,1.48869,10,7.0949,6,2865,130,14,6.5,0.045375,31.833333,4.083770
54,Gabriel Fernando de Jesus,FWD,Arsenal,1.31071,7,6.7020,11,2064,125,17,8.0,0.060562,22.933333,5.450581
74,Yoane Wissa,FWD,Brentford,0.93299,4,3.0141,7,1587,111,6,6.0,0.069943,17.633333,6.294896
78,Aleksandar Mitrović,FWD,Fulham,0.65529,2,7.5020,14,2010,107,14,7.5,0.053234,22.333333,4.791045
87,Julián Álvarez,FWD,Man City,1.32224,3,5.4620,9,1448,103,14,6.5,0.071133,16.088889,6.401934
